# Simple Agent Demo with Google ADK

This notebook demonstrates how to use the simple agent with custom tools using Google's Agent Development Kit (ADK) and LiteLLM.

In [2]:
# Install required packages
%pip install -q google-adk nest-asyncio

Note: you may need to restart the kernel to use updated packages.


## Setup

First, let's import the necessary modules.

In [3]:
import sys
sys.path.insert(0, '.')

# Enable nested event loops for Jupyter
import nest_asyncio
nest_asyncio.apply()

from simple_agent import run_agent, run_agent_async, simple_agent

/workspaces/agentx-hackathon-DC-Pros/.venv/lib/python3.11/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


## Check Agent Configuration

Let's verify the agent is configured correctly.

In [4]:
print(f"Agent Name: {simple_agent.name}")
print(f"Model: {simple_agent.model.model}")
print(f"Number of Tools: {len(simple_agent.tools)}")
print(f"Tool Names: {[tool.__name__ for tool in simple_agent.tools]}")

Agent Name: simple_agent
Model: openai/gemini-2.5-flash
Number of Tools: 2
Tool Names: ['calculator', 'text_analyzer']


In [5]:
# Create a session for this notebook
import uuid
from simple_agent import session_service, APP_NAME

# Generate a unique session ID for this notebook run
NOTEBOOK_USER_ID = "notebook_user"
NOTEBOOK_SESSION_ID = f"session_{uuid.uuid4().hex[:8]}"

# Create the session
await session_service.create_session(
    app_name=APP_NAME, 
    user_id=NOTEBOOK_USER_ID, 
    session_id=NOTEBOOK_SESSION_ID
)

print(f"✓ Session created: {NOTEBOOK_SESSION_ID}")

✓ Session created: session_94759532


## Example 1: Using the Calculator Tool

Ask the agent to perform calculations. The agent will use the calculator tool automatically.

In [6]:
response = run_agent("What is 45 multiplied by 23?", user_id=NOTEBOOK_USER_ID, session_id=NOTEBOOK_SESSION_ID)
print(f"\nFinal Answer: {response}")


>>> Query: What is 45 multiplied by 23?
[Tool Call] calculator(operation='multiply', a=45, b=23)
[Tool Result] 1035
<<< Response: 45 multiplied by 23 is 1035.


Final Answer: 45 multiplied by 23 is 1035.


## Example 2: Using the Text Analyzer Tool

Ask the agent to analyze text.

In [7]:
response = run_agent("How many words are in this sentence: 'The quick brown fox jumps over the lazy dog'?", user_id=NOTEBOOK_USER_ID, session_id=NOTEBOOK_SESSION_ID)
print(f"\nFinal Answer: {response}")


>>> Query: How many words are in this sentence: 'The quick brown fox jumps over the lazy dog'?
[Tool Call] text_analyzer(text='The quick brown fox jumps over the lazy dog...', analysis_type='word_count')
[Tool Result] {'word_count': 9}
<<< Response: There are 9 words in the sentence 'The quick brown fox jumps over the lazy dog'.


Final Answer: There are 9 words in the sentence 'The quick brown fox jumps over the lazy dog'.


## Example 3: Complex Calculation

Ask a question that requires multiple calculations.

In [8]:
response = run_agent("If I have 100 dollars and spend 25% of it, then add 50 dollars, how much do I have?", user_id=NOTEBOOK_USER_ID, session_id=NOTEBOOK_SESSION_ID)
print(f"\nFinal Answer: {response}")


>>> Query: If I have 100 dollars and spend 25% of it, then add 50 dollars, how much do I have?
[Tool Call] calculator(operation='multiply', a=100, b=0.25)
[Tool Result] 25.0
[Tool Call] calculator(operation='subtract', a=100, b=25)
[Tool Result] 75
[Tool Call] calculator(operation='add', a=75, b=50)
[Tool Result] 125
<<< Response: You would have 125 dollars.


Final Answer: You would have 125 dollars.


## Example 4: Simple Question (No Tools)

Ask a simple question that doesn't require any tools.

In [9]:
response = run_agent("What is the capital of France?", user_id=NOTEBOOK_USER_ID, session_id=NOTEBOOK_SESSION_ID)
print(f"\nFinal Answer: {response}")


>>> Query: What is the capital of France?


<<< Response: I cannot answer that question with the tools I have.


Final Answer: I cannot answer that question with the tools I have.


## Example 5: Multiple Operations

Test with division operation.

In [10]:
response = run_agent("What is 150 divided by 6?", user_id=NOTEBOOK_USER_ID, session_id=NOTEBOOK_SESSION_ID)
print(f"\nFinal Answer: {response}")


>>> Query: What is 150 divided by 6?
[Tool Call] calculator(operation='divide', a=150, b=6)
[Tool Result] 25.0
<<< Response: 150 divided by 6 is 25.


Final Answer: 150 divided by 6 is 25.


## Try Your Own Question

Now try asking your own questions! You can use verbose=False to hide the detailed output.

In [11]:
# Replace with your own question
my_question = "What is 25 plus 37?"

# With verbose output (shows tool calls)
response = run_agent(my_question, user_id=NOTEBOOK_USER_ID, session_id=NOTEBOOK_SESSION_ID, verbose=True)

# Or without verbose output (cleaner)
# response = run_agent(my_question, user_id=NOTEBOOK_USER_ID, session_id=NOTEBOOK_SESSION_ID, verbose=False)

print(f"\nFinal Answer: {response}")


>>> Query: What is 25 plus 37?
[Tool Call] calculator(operation='add', a=25, b=37)
[Tool Result] 62
<<< Response: 25 plus 37 is 62.


Final Answer: 25 plus 37 is 62.


## Using Async (Advanced)

If you need async functionality in notebooks, you can use `await` directly.

In [12]:
# In Jupyter/Colab, you can use await directly
response = await run_agent_async("What is 12 times 8?", user_id=NOTEBOOK_USER_ID, session_id=NOTEBOOK_SESSION_ID)
print(f"\nFinal Answer: {response}")


>>> Query: What is 12 times 8?
[Tool Call] calculator(operation='multiply', a=12, b=8)
[Tool Result] 96
<<< Response: 12 times 8 is 96.


Final Answer: 12 times 8 is 96.
